# **Project-Group-15**
Abhishek Sharma, Owen Pannucci, Krish Arulalan

Department of Computer Science, Rice University

as537@rice.edu, op21@rice.edu, ka72@rice.edu

NOTE: This code was executed on Google Colab with a T4 GPU.

In [ ]:
import transformers
from transformers import pipeline, logging
import torch
from google.colab import userdata
from huggingface_hub import login

## Primpt Construction
#### Combines deafult querries with OCR output and user queries

In [ ]:
def format_label_for_prompt(label_data, user_queries=None):

    nutrition_info = label_data

    default_queries = [
        'A brief summary of the nutritional profile.',
        'Any health risks or red flags?',
        'Dietary suitability.',
    ]

    if user_queries:
        default_queries.extend(user_queries)

    query_text = "\n".join(f"{i+1}. {q}" for i, q in enumerate(default_queries))

    user_message = f"""Here is the nutrition information:
{nutrition_info}

Provide:
{query_text}
"""
    return user_message


label_data = """
Ingredients: sugar, salt, artificial flavor,
peanut butter
Total Fat: 1.5g, 2%
Sodium: 2g, 2.5%
Sugars: 10g
Protein: 1g
Calories: 120
"""


extra_queries = [
"Does this food contain wheat?",
"Is it suitable for someone with peanut allergies?"
]

user_message = format_label_for_prompt(label_data, user_queries=extra_queries)

print(user_message)


Here is the nutrition information:

Ingredients: sugar, salt, artificial flavor, 
peanut butter
Total Fat: 1.5g, 2%
Sodium: 2g, 2.5%
Sugars: 10g
Protein: 1g
Calories: 120


Provide:
1. A brief summary of the nutritional profile.
2. Any health risks or red flags?
3. Dietary suitability.
4. Does this food contain wheat?
5. Is it suitable for someone with peanut allergies?



## Load Llama 3.2

In [ ]:
logging.set_verbosity(logging.ERROR)
login(token=userdata.get('secretName'))

model_id = "meta-llama/Llama-3.2-1B-Instruct" #3.3-70B too big :(
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## Prompt Engineering with Role Based Prompting

In [ ]:
#for llama 3.2
messages = [
    {"role": "system", "content": "You are a food, nutrition, and allergy expert. Your task is to analyze text extracted from a food nutrition label and answer user questions about nutritional content, ingredients, serving size, allergens, or other details present in the label. Base your answers solely on the provided text, without making assumptions or adding external knowledge.\n\nFor allergens: Only identify an allergen if it is explicitly listed in the ingredient list (e.g., 'milk', 'peanuts') or in a 'Contains' statement (e.g., 'Contains: wheat'). Do not assume an allergen is present based on vague terms like 'flour' or 'powder' unless the label specifies the source (e.g., 'wheat flour'). Do not assume cross-reactivity between allergens (e.g., do not equate a tree nut allergy with a peanut allergy unless explicitly stated).\n\nFor dietary suitability: Confirm suitability for specific diets (e.g., gluten-free, vegan) only if supported by the label. A product can be considered gluten-free if it lacks gluten-containing ingredients (e.g., wheat, barley, rye) in the ingredient list, but note potential cross-contamination risks unless explicitly labeled 'gluten-free'. For vegan suitability, ensure no animal-derived ingredients (e.g., milk, eggs) are listed. Do not speculate about suitability for unlisted dietary restrictions or processing-related contamination unless stated.\n\nProvide accurate, concise, and clear answers. If a question cannot be answered due to missing or ambiguous information, state so clearly and explain why. Avoid referencing unlisted ingredients, health risks, or nutritional concerns not explicitly mentioned in the provided text."},
    {"role": "user", "content": user_message},
]

outputs = pipeline(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]['generated_text'][-1]['content'])

**1. Brief Summary of the Nutritional Profile:**
The food contains 120 calories, 1.5g of total fat, 2g of sodium, and 10g of sugars. It is a low-calorie, low-fat, and low-sugar food.

**2. Health Risks or Red Flags:**
There are no health risks or red flags mentioned in the provided information.

**3. Dietary Suitability:**
This food is suitable for a gluten-free diet. It does not contain any gluten-containing ingredients, such as wheat, barley, or rye. However, it is essential to note that the label does not explicitly state that it is vegan or contains any animal-derived ingredients.

**4. Does this food contain wheat?**
No, this food does not contain wheat.

**5. Is it suitable for someone with peanut allergies?**
Yes, this food is suitable for someone with peanut allergies. It does not list peanut as an ingredient, and it does not have any statements that could indicate the presence of peanuts or other tree nuts.
